In [1]:
import numpy as np

In [40]:
#defining the matrices

Reply = np.array([
    [1, 5, 5, 0, 3],
    [2, 0, 5, 0, 3],
    [0, 0, 0, 0, 0],
    [2, 0, 0, 4, 0],
    [0, 4, 2, 0, 0]
])


Retweet = np.array([
    [0, 2, 4, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 3],
    [0, 2, 5, 0, 0]
])

Like = np.array([
    [2, 5, 1, 0, 3],
    [0, 0, 0, 0, 0],
    [0, 5, 0, 3, 3],
    [1, 0, 0, 4, 1],
    [0, 0, 9, 0, 0]
])

Post = np.array([2, 5, 10, 4, 3])


# Engagement Statistics

In [7]:
#Find the user who replies more than others.

print(f"the user is : User {np.argmax(np.sum(Reply, axis=1))}")

the user is : User 0


In [8]:
#Find the user who retweets more than others.
print(f"the user is : User {np.argmax(np.sum(Retweet, axis=1))}")

the user is : User 4


In [12]:
#Find users who replied or retweeted at least once.
Reply_Retweet_matrix = Reply + Retweet
interacations = Reply_Retweet_matrix.sum(axis=1)

users = [i for i in range(len(interacations)) if interacations[i]> 0]

print(f"users are : {users}")

users are : [0, 1, 3, 4]


In [14]:
#Display the number of likes each user has given to their own tweets.
diagnol = [int(Like[i][i]) for i in range(5)]

print(f"each user likes to him self : {diagnol}")

each user likes to him self : [2, 0, 0, 4, 0]


In [19]:
#Display the number of likes each user has given to tweets other than their own.
diagnol = [Like[i][i] for i in range(5)]
other_like = Like.sum(axis=1)-diagnol
print(f"number of likes user gives to others {other_like}")

number of likes user gives to others [ 9  0 11  2  9]


In [22]:
#Find users who gave likes to their own tweets more than they gave to others
diffrence = diagnol - other_like
selfish_users = [i for i in range(len(diffrence)) if diffrence[i]>0]
print(f"selfish users are : {selfish_users}")
print(f"thier number is : {len(selfish_users)}")

sefish users are : [3]
thier number is : 1


In [23]:
#Display the number of interactions each user has made.
all_interactions = Reply + Like + Retweet
print(f"users intercations : {all_interactions.sum(axis=1)}")

users intercations : [31 10 11 15 22]


# Interaction Statistics

In [41]:
#Display the maximum number of likes each user has received (exclude himself)
Likes_recived = Like - [Like[i][i] for i in range(Like.shape[0])] * np.eye(Like.shape[0])
max_likes_recived = Likes_recived.max(axis=0)
print(f"max number of likes recived from other users {max_likes_recived} ")

max number of likes recived from other users [1. 5. 9. 3. 3.] 


In [46]:
#Find users who have received likes for all their tweets by at least one person other than themselves.
print(f"users who have received likes for all their tweets by at least one person {max_likes_recived>=Post}")

users who have received likes for all their tweets by at least one person [False  True False False  True]


In [52]:
#Calculate the number of incoming edges
incoming_edges = (all_interactions > 0).sum(axis=0)
print(f"number of incoming_edge {incoming_edges}")

number of incoming_edge [3 3 3 2 4]


In [50]:
#Calculate the number of outgoing edges
outgoing_edges = (all_interactions > 0).sum(axis=1)
print(f"number of outgoing_edge {outgoing_edges}")

number of outgoing_edge [4 3 3 3 2]


In [56]:
#nodes where the number of incoming edges is equal to the number of outgoing edges.
nodes = outgoing_edges == incoming_edges
print(nodes)

[False  True  True False False]


In [64]:
#Display the likes they have given to each other.
Like[:, nodes][nodes,:]

array([[0, 0],
       [5, 0]])

# Popularity

In [68]:
I = np.eye(Reply.shape[0])
I1 = 1 - I

Reply_nodiag =Reply * I1
Like_nodiag = Like * I1
Retweet_diag = Retweet * I

Reply_nodiag, Retweet_diag , Like_nodiag

(array([[0., 5., 5., 0., 3.],
        [2., 0., 5., 0., 3.],
        [0., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0.],
        [0., 4., 2., 0., 0.]]),
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 5., 1., 0., 3.],
        [0., 0., 0., 0., 0.],
        [0., 5., 0., 3., 3.],
        [1., 0., 0., 0., 1.],
        [0., 0., 9., 0., 0.]]))

In [69]:
# For each interaction, 
# if the number of Replies received from a person is more than their Likes, we multiply the number of Replies by (-0.5). 
# If it is equal, we multiply their own Likes by (-0.25). 
# Otherwise, it has no effect (0).

Reply_score = np.where(Reply_nodiag > Like_nodiag, Reply_nodiag * (-0.5), np.where(Reply_nodiag == Like_nodiag, 0, Reply_nodiag * -0.25) )

Reply_score

array([[ 0. ,  0. , -2.5,  0. ,  0. ],
       [-1. ,  0. , -2.5,  0. , -1.5],
       [ 0. , -0. ,  0. , -0. , -0. ],
       [-1. ,  0. ,  0. ,  0. , -0. ],
       [ 0. , -2. , -0.5,  0. ,  0. ]])

In [70]:
# if someone retweets their own tweets, they will be penalized by (-0.25) for each retweet.

Retweet_score = -0.25 * Retweet_diag

In [73]:
# For each interaction, 
# if the number of Likes received from a person is greater than half of the tweets published, we multiply it by (1.5). 
# Otherwise, the score is the number of Likes.


Post_half = Post / 2
Post_repeated = np.ones(Like.shape) 
Post_repeated = Post_repeated * Post_half

Like_score = np.where(Like_nodiag > Post_repeated , Like_nodiag * (1.5) , Like_nodiag)

In [74]:
## Calculate the popularity score for each user for each interaction which is the average of the received scores.
detaild_popularity_score = (Reply_score + Retweet_score + Like_score)/3

detaild_popularity_score


array([[ 0.        ,  2.5       , -0.5       ,  0.        ,  1.5       ],
       [-0.33333333,  0.        , -0.83333333,  0.        , -0.5       ],
       [ 0.        ,  2.5       ,  0.        ,  1.5       ,  1.5       ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.33333333],
       [ 0.        , -0.66666667,  4.33333333,  0.        ,  0.        ]])

In [76]:
popularity_score = detaild_popularity_score.mean(axis=0)

popularity_score

array([-0.06666667,  0.86666667,  0.6       ,  0.3       ,  0.56666667])